<a href="https://colab.research.google.com/github/katrin2202/NLP/blob/main/Tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Токенизация слов

Самый простой способ токенизации предложения — применение внутри строк
пробелов в качестве разделителей слов. В языке Python для этого подходит метод
split из стандартной библиотеки, доступный для всех экземпляров объекта str,
а также для самого встроенного класса str

In [3]:
sentence = "Thomas Jefferson began building Monticello at the age of 26."
sentence.split()

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26.']

In [4]:
str.split(sentence)

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26.']

Для дальнейшей работы с текстом нам нужно преобразовать текст в числовый вектор. Эти вектор называется *унитарными*.

In [5]:
import numpy as np

`str.split()` самый быстрый токенизатор  
`vocab` - словарь уникальных токенов

In [8]:
token_sequence = str.split(sentence)
vocab = sorted(set(token_sequence))
', '.join(vocab)

'26., Jefferson, Monticello, Thomas, age, at, began, building, of, the'

`onehot_vectors` хранится отсортированый лексикографически слова
В цикле для каждого слова в предложении помечаем соответствующий столбец в словаре единицей

In [9]:
num_tokens = len(token_sequence)
vocab_size = len(vocab)
onehot_vectors = np.zeros((num_tokens, vocab_size), int)
for i, word in enumerate(token_sequence):
  onehot_vectors[i, vocab.index(word)] = 1
' '.join(vocab)

'26. Jefferson Monticello Thomas age at began building of the'

In [10]:
onehot_vectors

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Объект DataFrame отслеживает метки для всех столбцов, позволяя пометить
каждый столбец в нашей таблице токеном или словом, которое он представляет.

In [11]:
import pandas as pd

In [12]:
pd.DataFrame(onehot_vectors, columns=vocab)

,26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the
0,0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0
9,1,0,0,0,0,0,0,0,0,0


Уберем нули, чтобы было более наглядно

In [13]:
df = pd.DataFrame(onehot_vectors, columns=vocab)
df[df == 0] = " "
df

,26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the
0,,,,1,,,,,,
1,,1,,,,,,,,
2,,,,,,,1,,,
3,,,,,,,,1,,
4,,,1,,,,,,,
5,,,,,,1,,,,
6,,,,,,,,,,1
7,,,,,1,,,,,
8,,,,,,,,,1,
9,1,,,,,,,,,


Можем поместить токены в двоичный вектор, указывающий на наличие или отсутствие конкретного слова в определенном предложении. Векторное представление набора предложений можно проиндексировать для
отражения информации о том, какие слова в каком предложении используются.

In [14]:
sentence_bow = {}
for token in sentence.split():
  sentence_bow[token] = 1
sorted(sentence_bow.items())

[('26.', 1),
 ('Jefferson', 1),
 ('Monticello', 1),
 ('Thomas', 1),
 ('age', 1),
 ('at', 1),
 ('began', 1),
 ('building', 1),
 ('of', 1),
 ('the', 1)]

Попробуем еще более эффективную форму словаря — класс Series библиотеки
Pandas.

In [15]:
import pandas as pd
df = pd.DataFrame(pd.Series(dict([(token, 1) for token in sentence.split()])), columns=['sent']).T
df

,Thomas,Jefferson,began,building,Monticello,at,the,age,of,26.
sent,1,1,1,1,1,1,1,1,1,1


Рассмотрим токенизацию на нескольких предложениях

In [21]:
sentences = "Thomas Jefferson began building Monticello at the are of 26.\n"
sentences += "Construction was done mostly by local masons and carpenters.\n"
sentences += "He moved into the South Pavilion in 1770.\n"
sentences += "Turning Monticello into a neoclassical masterpiece was Jefferson's obsession."
corpus = {}
for i, sent in enumerate(sentences.split('\n')):
  corpus['sent{}'.format(i)] = dict((tok, 1) for tok in sent.split())
df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T
df[df.columns[:10]]

,Thomas,Jefferson,began,building,Monticello,at,the,are,of,26.
sent0,1,1,1,1,1,1,1,1,1,1
sent1,0,0,0,0,0,0,0,0,0,0
sent2,0,0,0,0,0,0,1,0,0,0
sent3,0,0,0,0,1,0,0,0,0,0


## Скалярное произведение

In [22]:
v1 = pd.np.array([1,2,3])
v2 = pd.np.array([2,3,4])
v1.dot(v2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


20

In [23]:
(v1 * v2).sum()

20

In [24]:
sum([x1*x2 for x1, x2 in zip(v1, v2)])

20

Подсчет пересечений количеств слов для 2 векторов мультимножеств слов

In [25]:
df = df.T
df.sent0.dot(df.sent1)

0

In [26]:
df.sent0.dot(df.sent2)

1

In [27]:
df.sent0.dot(df.sent3)

1

Видим, что у 0 и 2 или 0 и 3 есть общее слово. Давайте его найдем

In [28]:
[(k, v) for (k, v) in (df.sent0 & df.sent3).items() if v]

[('Monticello', 1)]

## Улучшение токенов


Создание токена.

Разбиваем предложение по пробелам или знакам препинания, встречающимся как минимум один  раз.

In [29]:
import re

In [30]:
sentence = "Thomas Jefferson began building Monticello at the are of 26."
tokens = re.split(r'[-\s.,;!?]+', sentence)
tokens

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'are',
 'of',
 '26',
 '']

Применим более сложные токенизаторы

In [31]:
!pip install regex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Для моделирования нашего простого примера токенизатора можно воспользоваться функцией RegexpTokenizer из библиотеки NLTK:

Этот токенизатор немного лучше, чем тот, который вы использовали изначально, поскольку игнорирует пробельные символы. Он также отделяет завершающие предложение знаки пунктуации от токенов, не содержащих других знаков препинания.

In [32]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+|$[0-9.]+|\S+')
tokenizer.tokenize(sentence)

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'are',
 'of',
 '26',
 '.']

Одним из лучших токенизаторов является Word Tokenizer Treebank, входящий в состав библиотеки NLTK. В нем множество общих правил для токенизации английских слов.
Он включает правила токенизации для английских сокращений.

In [33]:
from nltk.tokenize import TreebankWordTokenizer
sentence = "Monticello wasn't designated as UNESCO World Heritage Site until 1987."
tokenizer = TreebankWordTokenizer()
tokenizer.tokenize(sentence)

['Monticello',
 'was',
 "n't",
 'designated',
 'as',
 'UNESCO',
 'World',
 'Heritage',
 'Site',
 'until',
 '1987',
 '.']

## Сокращения

Токенизация неформального текста из социальных сетей, таких как Twitter и 
Facebook

In [34]:
from nltk.tokenize.casual import casual_tokenize
message = "RT @TJMonticello Best day everrrrrrr at Monticello. Awesommmmmmeeeeeeee day :*)"
casual_tokenize(message)

['RT',
 '@TJMonticello',
 'Best',
 'day',
 'everrrrrrr',
 'at',
 'Monticello',
 '.',
 'Awesommmmmmeeeeeeee',
 'day',
 ':*)']

In [35]:
casual_tokenize(message, reduce_len=True, strip_handles=True)

['RT',
 'Best',
 'day',
 'everrr',
 'at',
 'Monticello',
 '.',
 'Awesommmeee',
 'day',
 ':*)']

## Расширяем словарь n-граммами

первоначальный токенизатор 1-грамм:

In [38]:
sentence = "Thomas Jefferson began building Monticello at the age of 26."
pattern = re.compile(r"([-\s.,;!?])+")
tokens = pattern.split(sentence)
tokens = [x for x in tokens if x and x not in '- \t\n.,;!?']
tokens

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26']

токенизатор n-грамм из модуля nltk:

In [39]:
from nltk.util import ngrams
list(ngrams(tokens, 2))

[('Thomas', 'Jefferson'),
 ('Jefferson', 'began'),
 ('began', 'building'),
 ('building', 'Monticello'),
 ('Monticello', 'at'),
 ('at', 'the'),
 ('the', 'age'),
 ('age', 'of'),
 ('of', '26')]

In [40]:
list(ngrams(tokens, 3))

[('Thomas', 'Jefferson', 'began'),
 ('Jefferson', 'began', 'building'),
 ('began', 'building', 'Monticello'),
 ('building', 'Monticello', 'at'),
 ('Monticello', 'at', 'the'),
 ('at', 'the', 'age'),
 ('the', 'age', 'of'),
 ('age', 'of', '26')]

In [41]:
two_grams = list(ngrams(tokens, 2))
[" ".join(x) for x in two_grams]

['Thomas Jefferson',
 'Jefferson began',
 'began building',
 'building Monticello',
 'Monticello at',
 'at the',
 'the age',
 'age of',
 'of 26']

## Стоп-слова

In [42]:
stop_words = ['a', 'an', 'the', 'on', 'of', 'off', 'this', 'is']
tokens = ['the', 'house', 'is', 'on', 'fire']
tokens_without_stopwords = [x for x in tokens if x not in stop_words]
print(tokens_without_stopwords)

['house', 'fire']


Список стоп-слов NLTK

In [44]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
import nltk
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
len(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


179

In [46]:
stop_words[:7]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours']

Лист стоп-слов NLTK

In [51]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stop_words

In [52]:
len(sklearn_stop_words)

318

In [53]:
len(stop_words)

179

In [54]:
len(stop_words.union(sklearn_stop_words))

AttributeError: ignored

In [55]:
len(stop_words.intersection(sklearn_stop_words))

AttributeError: ignored

## Нормализация словаря

Выравнивание регистра

In [56]:
tokens = ['House', 'Visitor', 'Center']
normalized_tokens = [x.lower() for x in tokens]
print(normalized_tokens)

['house', 'visitor', 'center']


Стемминг

In [57]:
def stem(phrase):
  return ' '.join([re.findall('^(.*ss|.*?)(s)?$',
    word)[0][0].strip("'") for word in phrase.lower().split()])

In [58]:
stem('houses')

'house'

In [59]:
stem("Doctor House's calls")

'doctor house call'

In [61]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
' '.join([stemmer.stem(w).strip("'") for w in
 "dish washer's washed dishes".split()])

'dish washer wash dish'

## Лемматизация

In [62]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize("better")

[nltk_data] Downloading package wordnet to /root/nltk_data...


LookupError: ignored

In [ ]:
'better'
>>> lemmatizer.lemmatize("better", pos="a")
'good'
>>> lemmatizer.lemmatize("good", pos="a")
'good'
>>> lemmatizer.lemmatize("goods", pos="a")
'goods'
>>> lemmatizer.lemmatize("goods", pos="n")
'good'
>>> lemmatizer.lemmatize("goodness", pos="n")
'goodness'
>>> lemmatizer.lemmatize("best", pos="a")
'best'